## Apprentissage supervisé - k plus proches voisins

In [308]:
# Question 1


In [309]:
#Question 2 :

#Pour utiliser l'algo en régression plutôt qu'en classification et prédire cette fois une valeur
#on pourrait, au lieu de retourner la classe la plus fréquente, 
# retourner la moyenne des valeurs les plus fréquente

In [310]:
import pandas as pd


colnames = ["sepal_length","sepal_width", "petal_length", "petal_width","Class Label"]
Data = pd.read_csv("iris.data",header=None, names=colnames)


Data.head(-1) 

,sepal_length,sepal_width,petal_length,petal_width,Class Label
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
144,6.7,3.3,5.7,2.5,Iris-virginica
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica


In [311]:
print(Data.shape)
Data.describe()

(150, 5)


,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [312]:
Data[colnames[:-1]].iloc[0]

sepal_length    5.1
sepal_width     3.5
petal_length    1.4
petal_width     0.2
Name: 0, dtype: float64

In [313]:
import numpy as np

def distance(p1,p2): #distance euclidienne
    if len(p1) == len(p2):
        res_carre = 0
        for i in range(len(p1)):
            res_carre += ( p1[i] - p2[i] )**2
        return np.sqrt(res_carre)
    else:
        return None


distance(Data[colnames[:-1]].iloc[0],Data[colnames[:-1]].iloc[1])

0.5385164807134502

In [314]:
def knn(point, data,  k):
    distances_voisins = []
    
    for index, p in data.iterrows():
        dist = distance(point,p)
        distances_voisins.append((dist,index))
    
    distances_voisins = sorted(distances_voisins) #tri selon le 1er arg du tuple, donc dist
    res = [index for d,index in distances_voisins[:k]]
    return res #retourne les indices des k plus proches voisins de point

def prediction_Class(point,Data,k):
    indices_k_proches_voisins = knn(point,Data[colnames[:-1]],k)
    compt = {}
    for i in indices_k_proches_voisins:

        laClassDuVoisin = Data.iloc[i,-1]
        if laClassDuVoisin in compt.keys():
            compt[laClassDuVoisin] = compt[laClassDuVoisin] + 1
        else:
            compt[laClassDuVoisin] = 1
   
    # obtenir pourcentage
    for key, val in compt.items():
        compt[key] = round(val/k*100,2)
    
    # trier par ordre croissant de pourcentage
    compt = sorted(compt.items(), key=lambda t: t[1])
    return compt[-1] # retourner le dernier element : class,pourcentage

    


point = [6.2,	3.4,	5.4,	2.3]
prediction_Class(point,Data,500)

('Iris-setosa', 10.0)

In [335]:
def PredireNosIndividus(Data,k):
    X_predicted = []
    for i, p in Data.iterrows():
        pt = p.drop(labels='Class Label') # juste on a retier la classe label du point
        X_predicted.append(prediction_Class(pt,Data,k)[0])
    return Data['Class Label'].values.tolist(), X_predicted


def confusion(X_actual,X_predicted):
    go = {'x_Actual':   X_actual,
        'x_Predicted':X_predicted
        }
    df = pd.DataFrame(go, columns=['x_Actual','x_Predicted'])
    confusion_matrix = pd.crosstab(df['x_Actual'], df['x_Predicted'], rownames=['Actual'], colnames=['Predicted'])
    
    print(confusion_matrix)
    precision = 0
    for j in range(len(confusion_matrix.index)):
        precision += confusion_matrix.iloc[j][j]
    precision /= confusion_matrix.to_numpy().sum()
    return precision

X_actual,X_predicted = PredireNosIndividus(Data,3)
print("Précision :",confusion(X_actual,X_predicted))

Predicted        Iris-setosa  Iris-versicolor  Iris-virginica
Actual                                                       
Iris-setosa               50                0               0
Iris-versicolor            0               47               3
Iris-virginica             0                3              47
Précision : 0.96
